# Create AML environment

Consider having these resources created through a CI/CD pipeline

## Init

#### Packages

In [8]:
from azureml.core import Workspace
from azureml.core import Environment

#### Authentication

In [9]:
ws = Workspace.from_config()
ws

Workspace.create(name='paolt-ml-v2', subscription_id='fadb1e32-9c96-4180-be9d-1811f4687cca', resource_group='paolt-ml-v2')

## Create an environment


We are creating an AML Custom Environment based on a curated one (Sklearn-CPU)

In [5]:
# databook_env = Environment.from_conda_specification(name="DataBook-Env", file_path="./conda.yml")
databook_env = Environment(
    name="DataBook-Env",
    description="Custom environment for DataBook v-0.1",
    tags={"DataBook ver": "v-0.1"},
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220815.v1",
    version="0.2",
)

Prior to running this code it is necessary to build the *wheel* for the *DataBook* package. To do so, run the following from the command prompt:

```
cd databook_module
python setup.py bdist_wheel
```

In [ ]:
whl_path = "../databook_module/dist/databook-0.1-py3-none-any.whl"
whl_url = Environment.add_private_pip_wheel(workspace=ws, file_path=whl_path, exist_ok=True)
databook_env.python.conda_dependencies.add_pip_package(whl_url)


We now register the environment to AML as a custom environment, and we build the reative image

In [6]:
databook_env.register(ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220616.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "DataBook-Env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
 

In [7]:
databook_env.build(ws)

Building a non-registered environment is not supported. Registering environment.
